In [ ]:
#!python prepare_data_spleen.py

In [1]:
from cnn.model import build_net
from spleen_dataset.dataloader import SpleenDataloader, SpleenDataset, get_training_augmentation, get_validation_augmentation
from spleen_dataset.config import dataset_folder
from spleen_dataset.utils import get_split_deterministic, get_list_of_patients
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TensorBoard
import matplotlib.pyplot as plt
import random

import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#   try:
#     tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])
#   except RuntimeError as e:
#     print(e)

2022-11-20 00:20:47.948821: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-20 00:20:48.041685: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-20 00:20:48.041698: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-20 00:20:48.582221: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

In [2]:
gpus

[]

In [3]:
data_path = './data/Task09_Spleen_2D'

patch_size = (128, 128)
batch_size = 32
num_classes = 2
skip_slices = 1
stem_filters = 16
max_depth = 4

augmentation = get_training_augmentation(patch_size)

/home/guilherme/Downloads/git/segqnas/.venv/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:1149: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(


In [4]:
patients = get_list_of_patients(dataset_folder)
train_augmentation = get_training_augmentation(patch_size)
val_augmentation = get_validation_augmentation(patch_size)

train_patients, val_patients = get_split_deterministic(patients, fold=1, num_splits=5, random_state=0)

train_dataset = SpleenDataset(train_patients, only_non_empty_slices=True, skip_slices=skip_slices)
val_dataset = SpleenDataset(val_patients, only_non_empty_slices=True)

train_dataloader = SpleenDataloader(train_dataset, batch_size, train_augmentation)
val_dataloader = SpleenDataloader(val_dataset, batch_size, val_augmentation)

FileNotFoundError: [Errno 2] No such file or directory: './spleen_dataset/data/Task09_Spleen/imagesTr'

In [ ]:
len(train_dataset)

In [ ]:
len(train_dataloader)

In [ ]:
id = random.randint(0, len(train_dataset))
image, label = train_dataset[id]

print(image.shape)
print(label.shape)

fig = plt.figure(figsize=(10, 10))

ax1 = fig.add_subplot(2,2,1)
ax1.imshow(image)
ax2 = fig.add_subplot(2,2,2)
ax2.imshow(label)

In [ ]:
id = random.randint(0, len(train_dataloader))
images, labels = train_dataloader[id]

image = images[0]
label =labels[0]

print(image.shape)
print(label.shape)

fig = plt.figure(figsize=(10, 10))

ax1 = fig.add_subplot(2,2,1)
ax1.imshow(image)
ax2 = fig.add_subplot(2,2,2)
ax2.imshow(label)

In [ ]:
net_list = [
    'down_vgg_3', 
    'down_vgg_3',
    'down_vgg_3',
    'down_vgg_3',
    'non_vgg_3',
    'up_vgg_3',
    'up_vgg_3',
    'up_vgg_3',
    'up_vgg_3'
]

fn_dict = {
    'down_vgg_3': {'cell': 'DownscalingCell', 'block': 'VGGBlock', 'kernel': 3, 'prob': 1/3},
    'up_vgg_3':   {'cell': 'UpscalingCell',   'block': 'VGGBlock', 'kernel': 3, 'prob': 1/3},
    'non_vgg_3':  {'cell': 'NonscalingCell',  'block': 'VGGBlock', 'kernel': 3, 'prob': 1/3},
}

In [ ]:
model = build_net((*patch_size, 1), stem_filters, max_depth, num_classes, fn_dict, net_list)
model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
from IPython.display import Image

plot_model(model,
           to_file='model.png',
           show_shapes=True,
           show_layer_names=True,
           rankdir='TB'
            )
Image('model.png')

In [ ]:
history = model.fit(
    train_dataloader,
    validation_data=val_dataloader,
    epochs=50,
    callbacks=[
        EarlyStopping(
            monitor="val_loss", mode="min", verbose=1, patience=5
        ),
        # ModelCheckpoint(
        #     "best_model.h5",
        #     save_best_only=True,
        #     monitor="val_loss",
        #     mode="min",
        # ),
        # TensorBoard(log_dir="./logs")
    ],
)

In [ ]:
# plotting the dice coeff results (accuracy) as a function of the number of epochs
plt.figure()
plt.plot(history.history['gen_dice_coef'])
plt.plot(history.history['val_gen_dice_coef'])
plt.title('Model: Generalized Dice Coeficient')
plt.ylabel('Dice Coef')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# plotting the dice coeff results (loss function) as a function of the number of epochs
plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model: Generalized Dice Coeficient')
plt.ylabel('Dice Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper right')
plt.show()

In [ ]:
!tensorboard --logdir='./logs'